In [1]:
import os
import cv2
from time import time
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
import torch.utils as U
import torch.nn.functional as F
from torch.utils.data import Dataset, RandomSampler, DataLoader
from torch.nn import Sequential, Linear, Conv2d, MaxPool2d, BatchNorm2d, ReLU

from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.transforms as T

/home/wonje/anaconda3/envs/ML/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DIR_TRAIN = "./data/train/"
DIR_VAL = "./data/valid/"
DIR_TEST = "./data/test/"

In [3]:
classes = os.listdir(DIR_TRAIN)

train_count = 0
val_count = 0
test_count = 0

train_paths = []
val_paths = []
test_paths = []

for _class in os.listdir(DIR_TRAIN):
    for img in os.listdir(DIR_TRAIN + _class):
        train_paths.append(DIR_TRAIN + _class + '/' + img)
        train_count += 1
    for img in os.listdir(DIR_VAL + _class):
        val_paths.append(DIR_VAL + _class + '/' + img)
        val_count += 1
    for img in os.listdir(DIR_TEST + _class):
        test_paths.append(DIR_TEST + _class + '/' + img)
        test_count += 1

class_to_int = {classes[i] : i for i in range(len(classes))}
print(len(classes))
print(train_count)
print(val_count)
print(test_count)

100
12639
500
500


In [4]:
class ButterflyAndMothDataset(Dataset):
    def __init__(self, paths_list, class_to_int, transforms=None):
        super().__init__()
        self.paths_list = paths_list
        self.class_to_int = class_to_int
        self.transforms = transforms
        
    def __getitem__(self, idx):
        img_path = self.paths_list[idx]
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img /= 255.
        
        label = img_path.split("/")[-2]
        label = self.class_to_int[label]
        
        if self.transforms != None:
            img = self.transforms(img)
            
        return img, label
        
    def __len__(self):
        return len(self.paths_list)
    
def getTransform():
    return T.Compose([
        T.ToTensor(),
        T.Resize(299)
    ])

In [5]:
train_dataset = ButterflyAndMothDataset(train_paths, class_to_int, getTransform())
val_dataset = ButterflyAndMothDataset(val_paths, class_to_int, getTransform())
test_dataset = ButterflyAndMothDataset(test_paths, class_to_int, getTransform())

train_random_sampler = RandomSampler(train_dataset)
val_random_sampler = RandomSampler(val_dataset)
test_random_sampler = RandomSampler(test_dataset)

train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    sampler=train_random_sampler
)

val_data_loader = DataLoader(
    dataset=val_dataset,
    batch_size=50,
    sampler=val_random_sampler
)

test_data_loader = DataLoader(
    dataset=test_dataset,
    batch_size=50,
    sampler=test_random_sampler
)

In [6]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super().__init__()
        self.conv = Sequential(
            Conv2d(in_channels, out_channels, kernel_size, bias=False, **kwargs),
            BatchNorm2d(out_channels),
            ReLU()
        )
        
    def forward(self, x):
        x = self.conv(x)
        return x
    
class Stem(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = Sequential(
            BasicConv2d(3, 32, 3, stride=2, padding='valid'),
            BasicConv2d(32, 32, 3, padding='valid'),
            BasicConv2d(32, 64, 3, padding='same'),
        )
        self.branch3x3_conv = BasicConv2d(64, 96, 3, stride=2, padding='valid')
        self.branch3x3_max = MaxPool2d(3, stride=2)
        
        self.branch7x7a = Sequential(
            BasicConv2d(160, 64, 1),
            BasicConv2d(64, 96, 3, padding='valid'),
        )
        
        self.branch7x7b = Sequential(
            BasicConv2d(160, 64, 1),
            BasicConv2d(64, 64, (7,1), padding='same'),
            BasicConv2d(64, 64, (7,1), padding='same'),
            BasicConv2d(64, 96, 3, padding='valid'),
        )
        
        self.branch_pool_a = BasicConv2d(192, 192, 3, padding='valid', stride=2)
        self.branch_pool_b = MaxPool2d(3, stride=2)
        
    def forward(self, x):
        x = self.conv(x)
        x = torch.cat((self.branch3x3_conv(x), self.branch3x3_max(x)), 1)
        x = torch.cat((self.branch7x7a(x), self.branch7x7b(x)), 1)
        x = torch.cat((self.branch_pool_a(x), self.branch_pool_b(x)), 1)
        return x

class Inception_Resnet_A(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.branch1x1 = BasicConv2d(in_channels, 32, 1)
        self.branch3x3 = Sequential(
            BasicConv2d(in_channels, 32, 1),
            BasicConv2d(32, 32, 3, padding='same'),
        )
        self.branch5x5 = Sequential(
            BasicConv2d(in_channels, 32, 1),
            BasicConv2d(32, 48, 3, padding='same'),
            BasicConv2d(48, 64, 3, padding='same')
        )
        self.channel_expansion = Conv2d(128, 384, 1)
        self.bn = BatchNorm2d(384)
        self.relu = ReLU()
        
    def forward(self, x):
        x_shortcut = x
        x = torch.cat((self.branch1x1(x), self.branch3x3(x), self.branch5x5(x)), 1)
        x = self.channel_expansion(x)
        x = self.bn(x * 0.17 + x_shortcut)
        x = self.relu(x)
        return x

class Reduction_A(nn.Module):
    def __init__(self, in_channels, k, l, m, n):
        super().__init__()
        self.branchmax = MaxPool2d(3, stride=2)
        self.branch3x3 = BasicConv2d(in_channels, n, 3, stride=2)
        self.branch5x5 = Sequential(
            BasicConv2d(in_channels, k, 1, padding='same'),
            BasicConv2d(k, l, 3, padding='same'),
            BasicConv2d(l, m, 3, stride=2),
        )
        
    def forward(self, x):
        x = torch.cat((self.branchmax(x), self.branch3x3(x), self.branch5x5(x)), 1)
        return x
    
class Inception_Resnet_B(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.branch1x1 = BasicConv2d(in_channels, 192, 1)
        self.branch7x7 = Sequential(
            BasicConv2d(in_channels, 128, 1),
            BasicConv2d(128, 160, (1,7), padding='same'),
            BasicConv2d(160, 192, (7,1), padding='same'),
        )
        self.channel_expansion = Conv2d(384, 1152, 1)
        self.bn = BatchNorm2d(1152)
        self.relu = ReLU()
        
    def forward(self, x):
        x_shortcut = x
        x = torch.cat((self.branch1x1(x), self.branch7x7(x)), 1)
        x = self.channel_expansion(x)
        x = self.bn(x * 0.1 + x_shortcut)
        x = self.relu(x)
        return x
    
class Reduction_B(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.branchmax = MaxPool2d(3, stride=2)
        self.branch3x3a = Sequential(
            BasicConv2d(in_channels, 256, 1),
            BasicConv2d(256, 384, 3, stride=2),
        )
        self.branch3x3b = Sequential(
            BasicConv2d(in_channels, 256, 1),
            BasicConv2d(256, 288, 3, stride=2),
        )
        self.branch5x5 = Sequential(
            BasicConv2d(in_channels, 256, 1),
            BasicConv2d(256, 288, 3, padding='same'),
            BasicConv2d(288, 320, 3, stride=2),
        )
        
    def forward(self, x):
        x = torch.cat((self.branchmax(x), self.branch3x3a(x), self.branch3x3b(x), self.branch5x5(x)), 1)
        return x
    
class Inception_Resnet_C(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.branch1x1 = BasicConv2d(in_channels, 192, 1)
        self.branch3x3 = Sequential(
            BasicConv2d(in_channels, 192, 1),
            BasicConv2d(192, 224, (1,3), padding='same'),
            BasicConv2d(224, 256, (3,1), padding='same'),
        )
        self.channel_expansion = Conv2d(448, 2144, 1)
        self.bn = BatchNorm2d(2144)
        self.relu = ReLU()
        
    def forward(self, x):
        x_shortcut = x
        x = torch.cat((self.branch1x1(x), self.branch3x3(x)), 1)
        x = self.channel_expansion(x)
        x = self.bn(x * 0.2 + x_shortcut)
        x = self.relu(x)
        return x
    
class Inception_Resnet_V2(nn.Module):
    def __init__(self, A=10, B=20, C=10, k=256, l=256, m=384, n=384, num_classes=100, init_weights=True):
        super().__init__()
        blocks = []
        blocks.append(Stem())
        for _ in range(A):
            blocks.append(Inception_Resnet_A(384))
        blocks.append(Reduction_A(384, 256, 256, 384, 384))
        for _ in range(B):
            blocks.append(Inception_Resnet_B(1152))
        blocks.append(Reduction_B(1152))
        for _ in range(C):
            blocks.append(Inception_Resnet_C(2144))
        
        self.features = Sequential(*blocks)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout2d(0.2)
        self.linear = nn.Flatten()
        self.classifier = nn.Linear(2144, num_classes)
        
        if init_weights:
            self._intialize_weights()
            
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.dropout(x)
        x = self.linear(x)
        x = self.classifier(x)
        return x
        
    def _intialize_weights(self):
        for m in self.modules():
            if isinstance(m, Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if isinstance(m, BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            if isinstance(m ,Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Inception_Resnet_V2()
# summary(model, (3, 299, 299) ,device=device.type)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.001)

from torch.optim.lr_scheduler import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=.1, patience=10)

epochs = 100
model.to(device)
def calc_accuracy(pred, true):
    pred = F.softmax(pred, dim = 1)
    true = torch.zeros(pred.shape[0], pred.shape[1]).scatter_(1, true.unsqueeze(1), 1)
    acc = (true.argmax(-1) == pred.argmax(-1)).float().detach().numpy()
    acc = float((100 * acc.sum()) / len(acc))
    return round(acc, 4)

In [13]:
train_loss = []
train_acc = []

val_loss = []
val_acc = []

writer_train = SummaryWriter(f"./runs/Inception-Resnetv2/train")
writer_val = SummaryWriter(f"./runs/Inception-Resnetv2/val")

for epoch in range(epochs):
    
    start = time()
    train_epoch_loss = []
    train_epoch_acc = []
    
    val_epoch_loss = []
    val_epoch_acc = []
    
    for imgs, labels in tqdm(train_data_loader):
        
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        y_hat = model(imgs)
        train_epoch_acc.append(calc_accuracy(y_hat.cpu(), labels.cpu()))
        loss = criterion(y_hat, labels)
        loss.backward()
        optimizer.step()
        
        train_epoch_loss.append(loss.item())
        
        
    for imgs, labels in tqdm(val_data_loader):
        
        with torch.no_grad():
            imgs = imgs.to(device)
            labels = labels.to(device)

            y_hat = model(imgs)
            val_epoch_acc.append(calc_accuracy(y_hat.cpu(), labels.cpu()))

            loss = criterion(y_hat, labels)
            print(loss.item())
            val_epoch_loss.append(loss.item())

    train_epoch_loss = np.mean(train_epoch_loss)
    train_epoch_acc = np.mean(train_epoch_acc)
    
    val_epoch_loss = np.mean(val_epoch_loss)
    val_epoch_acc = np.mean(val_epoch_acc)
    
    train_loss.append(train_epoch_loss)
    train_acc.append(train_epoch_acc)
    val_loss.append(val_epoch_loss)
    val_acc.append(val_epoch_acc)
    
    writer_train.add_scalar("Loss", train_epoch_loss, epoch+1)
    writer_train.add_scalar("Accuracy", train_epoch_acc, epoch+1)
    writer_val.add_scalar("Loss", val_epoch_loss, epoch+1)
    writer_val.add_scalar("Accuracy", val_epoch_acc, epoch+1)
    
    end = time()
    
    print(f"Epoch {epoch} - {int(end-start)}s")
    print(f"Train || Loss: {round(train_epoch_loss, 4)}, Acc: {train_epoch_acc}")
    print(f"Valid || Loss: {round(val_epoch_loss, 4)}, Acc: {val_epoch_acc}")
    
writer_train.close()
writer_val.close()

 20%|████████▊                                   | 2/10 [00:00<00:01,  5.64it/s]

4.618396759033203
4.619017124176025


 40%|█████████████████▌                          | 4/10 [00:00<00:01,  5.60it/s]

4.606338024139404
4.574936389923096


 60%|██████████████████████████▍                 | 6/10 [00:01<00:00,  5.65it/s]

...

4.583884239196777
4.6192145347595215


 80%|███████████████████████████████████▏        | 8/10 [00:01<00:00,  5.62it/s]

4.617909908294678
4.620747089385986


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  5.57it/s]

4.61623477935791
4.612863540649414
Epoch 99 - 122s
Train || Loss: 4.5992, Acc: 1.479685569620253
Valid || Loss: 4.6118, Acc: 1.0
